In [ ]:
# Установка зависимостей
!pip install reedsolo bitarray matplotlib

import numpy as np
from collections import Counter
from bitarray import bitarray
from reedsolo import RSCodec, ReedSolomonError
import matplotlib.pyplot as plt
from google.colab import files

# Загрузка файла
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

# === 1. Построение таблицы Шеннона-Фано ===
def build_tree(freqs):
    items = sorted(freqs.items(), key=lambda x: x[1], reverse=True)

    def shannon_fano(items, code=''):
        if len(items) == 1:
            return {items[0][0]: code}
        total = sum(freq for _, freq in items)
        acc = 0
        split = 0
        for i, (_, freq) in enumerate(items):
            acc += freq
            if acc >= total / 2:
                split = i + 1
                break
        left = shannon_fano(items[:split], code + '0')
        right = shannon_fano(items[split:], code + '1')
        left.update(right)
        return left

    return shannon_fano(items)

freqs = Counter(text)
code_table = build_tree(freqs)

def encode_shannon(text, code_table):
    return ''.join(code_table[ch] for ch in text)

# Кодирование текста
encoded_bin = encode_shannon(text, code_table)
bit_arr = bitarray(encoded_bin)
byte_data = bit_arr.tobytes()

# === 2. Кодирование Рида-Соломона (RS(255,223)) ===
rs = RSCodec(32)  # 255-223 = 32

def split_blocks(data, block_size=223):
    return [data[i:i+block_size] for i in range(0, len(data), block_size)]

blocks = split_blocks(byte_data)
rs_encoded_blocks = [rs.encode(block) for block in blocks]
rs_encoded = b''.join(rs_encoded_blocks)

# === 3. Канал с AWGN ===
def awgn_channel(data_bytes, snr_db):
    snr_linear = 10 ** (snr_db / 10)
    signal_power = 1
    noise_power = signal_power / snr_linear
    noisy_bytes = bytearray()

    for byte in data_bytes:
        noisy_byte = 0
        for i in range(8):
            bit = (byte >> (7 - i)) & 1
            noise = np.random.normal(0, np.sqrt(noise_power))
            received = bit + noise
            bit_received = int(received >= 0.5)
            noisy_byte |= (bit_received << (7 - i))
        noisy_bytes.append(noisy_byte)

    return bytes(noisy_bytes)

# === 4. Декодирование RS + Шеннон-Фано ===
def decode_blocks(data, block_size=255):
    blocks = split_blocks(data, block_size)
    decoded = []
    for block in blocks:
        try:
            decoded_data = rs.decode(block)
            if isinstance(decoded_data, str):  # защита от неожиданного типа
                decoded_data = decoded_data.encode('utf-8')
            decoded.append(decoded_data)
        except ReedSolomonError:
            decoded.append(b'?' * 223)
    return b''.join(decoded)


def decode_shannon(decoded_bytes, code_table):
    rev_table = {v: k for k, v in code_table.items()}
    bits = bitarray()
    bits.frombytes(decoded_bytes)
    bin_str = bits.to01()

    decoded = []
    code = ''
    for b in bin_str:
        code += b
        if code in rev_table:
            decoded.append(rev_table[code])
            code = ''
    return ''.join(decoded)

# === 5. Оценка ошибок ===
def ber(original, recovered):
    min_len = min(len(original), len(recovered))
    diff = sum(o != r for o, r in zip(original[:min_len], recovered[:min_len]))
    return diff / min_len if min_len > 0 else 1.0

snr_range = range(1, 11)
ber_values = []

for snr in snr_range:
    print(f"SNR = {snr} dB...")
    noisy = awgn_channel(rs_encoded, snr)
    decoded_bytes = decode_blocks(noisy)
    recovered_text = decode_shannon(decoded_bytes, code_table)
    ber_score = ber(text, recovered_text)
    print(f"BER: {ber_score:.5f}")
    ber_values.append(ber_score)

# === 6. Построение графика ===
plt.plot(snr_range, ber_values, marker='o')
plt.xlabel("SNR (дБ)")
plt.ylabel("Bit Error Rate (BER)")
plt.title("BER от SNR при передаче с RS + Shannon-Fano")
plt.grid(True)
plt.show()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.8/314.8 kB 7.8 MB/s eta 0:00:00
